Azimuthal Averages
=============


**Summary:**    Azimuthal averages of requested output variables.  Each output variable is stored as a 2-D function, $q_i(r, \theta, t)$, such that

$q_i(r, \theta, t) = \frac{1}{2\pi}\int_0^{2\pi} f_i(r,\theta,\phi,t) d\phi $

for each desired output $f_i$.

**Subdirectory:**  AZ_Avgs

**main_input prefix:** azavg

**Python Class:** AZ_Avgs

**Additional Namelist Variables:**  
None

Azimuthally-Averaged outputs are particularly useful for examining a system's mean flows (i.e., differential rotation and meridional circulation). 

Examining the *main_input* file, we see that the following output values have been denoted for the Azimuthal Averages:


| Menu Code  | Description |
|------------|-------------|
| 1          | Radial Velocity |
| 2          | Theta Velocity |
| 3          | Phi Velocity  |
| 201        | Radial Mass Flux |
| 202        | Theta Mass Flux |
| 501        | Temperature Perturbation |



In the example that follows, we demonstrate how to plot azimuthal averages, including how to generate streamlines of mass flux.   Note that for Boussinesq models, our velocity and mass flux fields are identical and so quantity codes 1 and 2 can be used for mass flux.  This is not the case when running an anelastic simulation, when 201 and 202 need to be used..

We begin with the usual preamble and also import two helper routines used for displaying azimuthal averages.

Examining the data structure, we see that the vals array is dimensioned to account for latitudinal variation, and that we have new attributes costheta and sintheta used for referencing locations in the theta direction.

In [ ]:
from rayleigh_diagnostics import AZ_Avgs, build_file_list, plot_azav, streamfunction
import matplotlib.pyplot as plt
import pylab
import numpy
import warnings
warnings.filterwarnings("ignore")

model_type = 1 # 1 for anelastic example, 2 Boussinesq example
base_dir = '/home/nfeatherstone/runs/cig_vis/'
font_size=14     # Font size for plot labels 

if (model_type == 1):
    model_dir = base_dir+'anelastic/'
    # Define some units for plotting purposes
    eunits = r'(erg cm$^{-3}$)'                # energy density
    tunits = '(s)'                             # time
    vunits = r'(cm s$^{-1}$)'                  # velocity
    dunits = '(cm)'                            # distance
    thermal_label = r' Specific Entropy '      # specific entropy
    thermal_units = r'(erg g$^{-1}$ K$^{-1}$)' # specific entropy
    lunits = r'(erg s$^{-1}$)'                 # energy / time (luminosity)
    funits = '(nHz)'                           # Frequency units
    mfunits = '(g cm$^{-2}$ s$^{-1}$)'
    
    # Next, we set some timestep ranges for plotting purposes
    imin = 0             # minimum iteration number to process for time series
    imax = 10000000      # maximum iteration number to process for time series
    
if (model_type == 2):
    model_dir = base_dir+'Boussinesq/'
    # Define some units for plotting purposes
    eunits = ''                          # energy density
    tunits = '(viscous diffusion times)' # time
    vunits = ''                          # velocity
    dunits = ''                          # distance
    thermal_label = ' Temperature ' 
    thermal_units = '' 
    lunits = ''                          # energy / time (luminosity)
    funits = ''                          # Frequency units
    mfunits = ''
    
    # Next, we set some timestep ranges for plotting purposes
    imin = 0             # minimum iteration number to process for time series
    imax = 10000000      # maximum iteration number to process for time series
    

files = build_file_list(imin,imax,path=model_dir+'AZ_Avgs')
az = AZ_Avgs(files[0],path='')
help(az)

***
Before creating our plots, let's time-average.  This works similarly to the Shell_Avgs class.

In [ ]:
az = AZ_Avgs(files,path='')
azavg = az.vals

lut = az.lut

vphi = azavg[:,:,lut[3],0]
rhovr = azavg[:,:,lut[201],0]
rhovtheta = azavg[:,:,lut[202],0]
temperature = azavg[:,:,lut[501],0]
radius = az.radius
costheta = az.costheta
sintheta = az.sintheta

nr = az.nr
ntheta=az.ntheta


Before we render, we need to do some quick post-processing:
1. Remove the spherical mean temperature from the azimuthal average.
2. Convert v_phi into omega
3. Compute the magnitude of the mass flux vector
4. Compute stream function associated with the mass flux field

In [ ]:
#Subtrace the ell=0 component from temperature at each radius
for i in range(nr):
    temperature[:,i]=temperature[:,i] - numpy.mean(temperature[:,i])

#Convert v_phi to an Angular velocity
omega=numpy.zeros((ntheta,nr))
for i in range(nr):
    omega[:,i]=vphi[:,i]/(radius[i]*sintheta[:])

# If model_type = 1, convert omega to nHz
if (model_type == 1):
    omega = omega/2/numpy.pi*1e9
    
#Generate a streamfunction from rhov_r and rhov_theta
psi = streamfunction(rhovr,rhovtheta,radius,costheta,order=0)
#contours of mass flux are overplotted on the streamfunction PSI
rhovm = numpy.sqrt(rhovr**2+rhovtheta**2)*numpy.sign(psi)    

Finally, we render the azimuthal averages.  

In [ ]:
#   Plot a single row of 3 images 
#   Spacing is default spacing set up by subplot

figdpi=300
sizetuple=(5.5*3,3*3)


tsize = 20     # title font size
cbfsize = 10   # colorbar font size
fig, ax = plt.subplots(ncols=3,figsize=sizetuple,dpi=figdpi)
plt.rcParams.update({'font.size': font_size})

#temperature
units = thermal_units
plot_azav(fig,ax[0],temperature,radius,costheta,sintheta,mycmap='RdYlBu_r',boundsfactor = 1, 
          boundstype='rms', units=units, fontsize = cbfsize)
ax[0].set_title(thermal_label,fontsize=tsize)

#Differential Rotation
units = funits
plot_azav(fig,ax[1],omega,radius,costheta,sintheta,mycmap='RdYlBu_r',boundsfactor = 0.5, 
          boundstype='rms', units=units, fontsize = cbfsize)
ax[1].set_title(r'$\Omega-\Omega_\mathrm{frame}$',fontsize=tsize)

#Mass Flux
units = mfunits
plot_azav(fig,ax[2],psi,radius,costheta,sintheta,mycmap='RdYlBu_r',boundsfactor = 1.5, 
          boundstype='rms', units=units, fontsize = cbfsize, underlay = rhovm)
ax[2].set_title('Mass Flux',fontsize = tsize)

saveplot=False
savefile='AZ_Avgs.pdf'
if (saveplot):
    p.savefig(savefile)  
else:
    plt.show()
